In [13]:
# download image/captcha
#img = driver.find_element_by_xpath(".//*[@id='trRandom3']/td[2]/img")
#img = driver.find_element_by_xpath(".//*img[2]")


In [1]:
from selenium import webdriver
import urllib.request
from PIL import Image
import cv2 
import mechanize

import requests
from bs4 import BeautifulSoup

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
URL = 'https://servicos.receita.fazenda.gov.br/Servicos/certidao/CNDConjuntaInter/InformaNICertidao.asp?tipo=2'

In [6]:
driver = webdriver.Chrome()

In [7]:
driver.get(URL)

In [8]:
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
tags = soup.findAll('img')

In [9]:
def get_captcha(driver, element, path):
    # now that we have the preliminary stuff out of the way time to get that image :D
    location = element.location
    size = element.size
    # saves screenshot of entire page
    driver.save_screenshot(path)

    # uses PIL library to open image in memory
    image = Image.open(path)

    left = location['x']
    top = location['y'] 
    right = location['x'] + size['width']
    bottom = location['y'] + size['height'] 

    
    image = image.crop((left, top, right, bottom))  # defines crop points
    image.save(path, 'png')  # saves new cropped image
    return(image)


In [10]:
img = driver.find_element_by_id('imgCaptchaSerpro')

img.location
img.size

In [16]:
capt = get_captcha(driver, img, f"data/sample.png")

In [18]:
import cv2

# carrega a imagem colorida e transforma ela em uma matriz do numpy
image = cv2.imread('data/sample.png')
# calcula o pixel com menor intensidade
# inicia a cor com o maior valor possivel
color = 255 * 3

# pega as dimensoes da imagem
(rows, cols, depth) = image.shape


In [19]:
for row in range(rows):
    for col in range(cols):

        # acessa um pixel da imagem
        pixel     = image[row][col]
        (r, g, b) = pixel.tolist()

        if color > (r + g + b):
            color = r + g + b

In [27]:
image2 = image.copy()

In [28]:
for row in range(rows):
    for col in range(cols):
        pixel = image[row][col]
        (r, g, b) = pixel.tolist()
        c = (r + g + b)

        # pinta de branco o que nao for letra
        if c != color:
            image2[row][col] = (255, 255, 255)
        # pinta de preto o que for letra
        else:
            image2[row][col] = (0, 0, 0)

In [29]:
image

array([[[ 72,  72,  72],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [250, 250, 250],
        [250, 250, 250],
        [250, 250, 250]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[  0,   0,   0],
        [ 96,  96,  96],
        [191, 191, 191],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[118, 118, 118],
        [  0,   0,   0],
        [  8,   8,   8],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[242, 242, 242],
        [ 40,  40,  40],
        [  0,   0,   0],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[250, 250, 250],
        [199, 199, 199],
        [  8,   8,   8],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [30]:
img = Image.fromarray(image,'RGB')
#img.save('my.png')
img.show()

In [31]:
def recursive_bounding(image, row, col, letter_color):

    min_x = col
    min_y = row

    max_x = col
    max_y = row

    (rows, cols, depth) = image.shape

    for r in range(    max(0, row - 1), min(row + 2, rows)):
        for c in range(max(0, col - 1), min(col + 2, cols)):
            if r == row and c == col:
                continue

            (r1, g1, b1) = image[r][c]
            t = (r1 + g1 + b1)

            # vizinho nao pertence a letra, ignora ele
            if t != letter_color:
                continue

            # apaga esse pixel
            # nao salvamos ele como branco pq precisaremos dele
            # no proximo processamento
            image[r][c] = (20, 20, 20)

            points = recursive_bounding(image, r, c, letter_color)
            (min_x1, max_x1, min_y1, max_y1) = points;

            min_x = min(min_x, min_x1, c)
            max_x = max(max_x, max_x1, c)
            min_y = min(min_y, min_y1, r)
            max_y = max(max_y, max_y1, r)

    return (min_x, max_x, min_y, max_y)

In [61]:
cols

180

In [62]:
image[0][0].to_list()

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

In [83]:
boxes = []
letter_color = 0

for row in range(rows):
    for col in range(cols):
        pixel     = image[row][col]
        (r, g, b) = pixel.tolist();
        t = (r + g + b)

        # nao eh letra, ignora o pixel
#        if t != letter_color:
#            continue

        # calcula a bounding box dessa letra
        box = recursive_bounding(image, row, col, letter_color)

        # salva o bounding box numa lista de boxes
        boxes.append(box)

boxes.sort()

<ipython-input-31-04c60363d4ca>:17: RuntimeWarning: overflow encountered in ubyte_scalars
  t = (r1 + g1 + b1)


In [84]:
t

765

In [67]:
letters = []

for box in boxes:
    (col0, col1, row0, row1) = box

    letter = image[row0:row1 + 1, col0:col1 + 1]
    letters.append(letter)

In [68]:
import os
import glob
import collections

def load_templates():
    # filtra os arquivos .png
    files = glob.glob('templates/*.png')

    # dicionario com as imagens
    # chave = letra
    # valor = lista de imagem
    templates = collections.defaultdict(list)

    for file in files:
        f = os.path.basename(file)

        # o primeiro caracter do nome do arquivo eh a letra
        # correspondente
        letter = f[0]

        # carrega a imagem
        img = cv2.imread(file)

        # muda a imagem pra tons de cinza
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # binariza a imagem
        (_, img) = cv2.threshold(img, 245, 255, cv2.THRESH_BINARY)

        # uma letra pode ter varios templates
        templates[letter].append(img)

    return templates

In [69]:
letters[0].shape

(1, 1, 3)

In [70]:
def break_captcha(letters, templates):
    captcha = ''
    unrec   = 0

    for letter in letters:

        (rows, cols, depth) = letters.shape
        best_error  = rows * cols
        best_letter = ''

        # copia a imagem, transformando de RGB para tons de cinza
        gray = cv2.cvtColor(letter, cv2.COLOR_RGB2GRAY)

        # binariza a imagem
        (_, blackwhite) = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY);

        for letter in templates:
            for template in templates[letter]:

                # se a imagem for de tamanho diferente do template
                # ignora o template
                if template.shape != blackwhite.shape:
                    continue

                # calculamos a diferenca entre as 2 imagens binarias
                xor = template ^ blackwhite

                errors = xor.sum()

                if errors < best_error:
                    best_error  = errors
                    best_letter = letter

                    # match perfeito
                    if errors == 0:
                        break;

            # match perfeito
            if best_match == 0:
                break;

        # nao casou exatamente com nenhuma letra
        # provavelmente eh uma nova letra
        if best_error != 0:
            cv.SaveImage("image_%d.png" % (unrec), gray)
            unrec = unrec + 1

        captcha = captcha + best_letter

    return captcha

In [78]:
img = Image.fromarray(letters[10], 'RGB')
#img.save('my.png')
img.show()

In [72]:
img.show()

In [73]:
letters[-2]

array([[[255, 255, 255]]], dtype=uint8)

In [74]:
templates = load_templates()

In [75]:
letters.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
 break_captcha(letters[0], templates)

In [ ]:
gray = cv2.cvtColor(letters[1], cv2.COLOR_RGB2GRAY)

In [ ]:
gray.show()

In [82]:
img = Image.fromarray(image, 'RGB')
#img.save('my.png')
img.show()